### download the last month of the CDS raw forecasts 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pathlib
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [4]:
HOME = pathlib.Path.home()

In [5]:
CWD = pathlib.Path.cwd()

In [6]:
CWD

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/notebooks/C3S')

### import local functions for the downloading of the C3S forecasts 

In [7]:
sys.path.append('../..')

In [8]:
from ICU_Water_Watch import C3S, domains

### show the predefined domains 

In [9]:
domains.domains

{'Tropical_Pacific': [140, 220, -25, 25],
 'SW_Pacific': [172.5, 190, -22.5, -12],
 'Fiji': [175, 183, -21, -15],
 'NZ': [161, 181, -50, -30],
 'Pacific': [140, 240, -50, 25],
 'C3S_download': [100, 240, -50, 30],
 'Water_Watch': [120, 240, -38, 22.5]}

### provider (always CDS for now)

In [10]:
provider = 'CDS'

### variable name

In [11]:
var_name = 'tprate'

### list of GCMs 

In [12]:
list_GCMs = ['ECMWF','UKMO','METEO_FRANCE','CMCC','DWD', 'NCEP', 'JMA', 'ECCC']

### lag in months (if need to download older forecasts)

In [13]:
lag = 0 

### get today's date 

In [14]:
date = datetime.utcnow()

### apply lag 

In [15]:
date = date - relativedelta(months=lag)

In [16]:
print(f"will download the forecasts issued in {date:%B %Y}")

will download the forecasts issued in September 2021


### path to download the realtime forecasts 

In [17]:
gcm_path = pathlib.Path(f'/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts')

In [18]:
if not(gcm_path.exists()): 
    gcm_path.mkdir(parents=True)

### use dask to parallelize the download of the forecasts 

In [19]:
list_GCMs

['ECMWF', 'UKMO', 'METEO_FRANCE', 'CMCC', 'DWD', 'NCEP', 'JMA', 'ECCC']

In [20]:
downloaded_files = []

for GCM in list_GCMs:
    
    args = dict(GCM=GCM,
    varname=var_name,
    year=date.year,
    month=date.month,
    leadtimes=[1, 2, 3, 4, 5, 6],
    opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
    domain=domains.domains['C3S_download'],
    file_format='netcdf',
    level='surface',
    max_retry=3)
    
    downloaded_files.append(delayed(C3S.download)(**args))


In [21]:
with ProgressBar():
    downloaded_files = compute(downloaded_files)

[                                        ] | 0% Completed |  0.0s
/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/CMCC/TPRATE/ensemble_seas_forecasts_tprate_from_2021_09_CMCC.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/JMA/TPRATE/ensemble_seas_forecasts_tprate_from_2021_09_JMA.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/NCEP/TPRATE/ensemble_seas_forecasts_tprate_from_2021_09_NCEP.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/UKMO/TPRATE/ensemble_seas_forecasts_tprate_from_2021_09_UKMO.netcdf exists already on disk, skipping download and returning path

/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/METEO_FRANCE/TPRATE/ensemble_seas_forecasts_tprate_from_2021_09_METEO_FRANCE.netcdf exists already on disk, skipp

In [22]:
# for GCM in list_GCMs: 
    
#     f = C3S.download(
#     GCM=GCM,
#     varname=var_name,
#     year=date.year,
#     month=date.month,
#     leadtimes=[1, 2, 3, 4, 5, 6],
#     opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
#     domain=domains.domains['C3S_download'],
#     file_format='netcdf',
#     level='surface',
#     max_retry=3,
#     )   
#     lfiles.append(f)

### check dimensions and return the total number of members in the C3S MME 

In [23]:
import xarray as xr

In [24]:
members = []
for f in downloaded_files[0]: 
    dset = xr.open_dataset(f)
    members.append(dset.dims['number'])
    print(dset.dims)
    if 'system' in dset.dims:
        print(dset['system'])
    

Frozen({'longitude': 141, 'latitude': 81, 'number': 51, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 62, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 51, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 50, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 50, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 124, 'time': 5})
Frozen({'longitude': 57, 'latitude': 33, 'number': 78, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 10, 'time': 5, 'system': 2})
<xarray.DataArray 'system' (system: 2)>
array([1, 2], dtype=int32)
Coordinates:
  * system   (system) int32 1 2
Attributes:
    long_name:  system


In [25]:
import numpy as np 

In [26]:
np.array(members).sum()

476